# Y21tenX Ingest

In [ ]:
# Initialize
import pandas as pd
import numpy as np
import math
import time
import json
import h5py

# File serializer
def serial(pad): # generate a zero padded serial number string
    sn_file = 'SN.txt'
    f = open(sn_file, "r")
    n = int(f.read())    
    f = open(sn_file, "w")
    f.write(str(n+1))
    f.close()
    return str(n).zfill(pad)

# Make some choices
dataset = 'Y21tenX'
geneset = 'nMod' # 'nMod' (783 neuromodulatory) or 'NP' (Canonical NPP and NP-GPCR) or 'PC' (all protein-coding)
#regions_common = ['VIS', 'AUD', 'SSp', 'MOp', 'AI', 'ACA', 'RSP', 'TEa-PERI-ECT', 'HIP']

# Decomment one of the following four options
regions = ['Union'] # aggregate of all regions
#regions = regions_common # iterate through all common regions
#regions = ['ACA', 'HIP', 'MOp', 'SSp', 'VIS'] # iterate through most populous common regions
#regions = ['MOp'] # single most populous of common regions

# Check for possibly impossible memory requirement
if geneset == 'PC' and regions == 'Union': print('*** BEWARE OF EXCESS MEMORY USAGE! ***')

# Set resource file paths
read_path = '/Users/stephens/Primary_scRNAseq_Data/' + dataset + '/expression_matrix.hdf5'
print(' Expression data read file:', read_path)
gene_sifter_path = 'Resources/All_mouse_'+geneset+'_genes.csv'
print('          Gene sifter file:', gene_sifter_path)
pc_gene_path = 'Resources/All_mouse_PC_genes.csv'
print('  Protein-coding gene file:', pc_gene_path)


In [ ]:
# Create annotation dataframe
anno_file = '/Users/stephens/Primary_scRNAseq_Data/Y21tenX/metadata.csv.xls'
# Create annotation import dictionary and sifter
anno_import_dict = {'class_label': 'class',
                    'class_order': 'class_order',
                    'class_color': 'class_color',
                    'neighborhood_label': 'neighborhood',
                    'neighborhood_id': 'neighborhood_order',
                    'neighborhood_color': 'neighborhood_color',
                    'subclass_label': 'subclass',
                    'subclass_order': 'subclass_order',
                    'subclass_color': 'subclass_color',
                    'cluster_label': 'cluster',
                    'cluster_order': 'cluster_order',
                    'cluster_color': 'cluster_color',
                    'region_label': 'region',
                    'region_id': 'region_order',
                    'region_color': 'region_color',
                    'sample_name': 'sample_name'}
anno_sifter = list(anno_import_dict.keys())

# Import and rename annotation columns
annos = pd.read_csv(anno_file, index_col='sample_name', usecols=anno_sifter, low_memory=False)
annos = annos.rename(columns=anno_import_dict).reset_index().iloc[:, ::-1]

annos_list = list(anno_import_dict.values())
annos = annos[annos_list]
annos.set_index('sample_name', inplace=True)


In [ ]:
# Initialize annotation support dict and metadata items list
support_dict = {}
write_annots = ['type','region','class','neighborhood','subclass','cluster',
                'region_color','class_color','neighborhood_color','subclass_color','cluster_color',
                'genes_mapped','counts_sum','ordinal']
support_dict.update({'annotation_labels': write_annots})
drop_annots = ['class_order','neighborhood_order','subclass_order','cluster_order','region_order','sample_name']
type_dict = {'Glutamatergic': '01 Neuronal', 'GABAergic': '01 Neuronal', 'Non-Neuronal': '02 Non-Neuronal'}


In [ ]:
# Ad hoc list fix for missing genes

missing_pcs = ['Adrbk1','Adrbk2','Bai1','Bai2','Bai3','Cd97','Dpcd','Eltd1','Emr1','Emr4','Gpr110','Gpr111',
               'Gpr112','Gpr113','Gpr114','Gpr115','Gpr116','Gpr123','Gpr124','Gpr125','Gpr126','Gpr128',
               'Gpr133','Gpr31c','Gpr52','Gpr56','Gpr64','Gpr97','Gpr98','Gucy1a3','Gucy1b3','Lphn1',
               'Prkaca-ps1','Prkcdbp','Rp2h','Scn2a1','Tctn2']
support_dict.update({'missing_pcs': missing_pcs})

# Read and fix list of protein-coding genes
pc_genes = list(pd.read_csv(pc_gene_path)['Symbol'])
print(len(pc_genes))
pc_genes = [x for x in pc_genes if x not in missing_pcs]
print(len(pc_genes))

# Read and fix list of sifter genes
gene_list = list(pd.read_csv(gene_sifter_path)['Symbol'])
gene_list = [x for x in gene_list if x not in missing_pcs]
print(len(gene_list))


In [ ]:
# Open the HDF5 file
with h5py.File(read_path, 'r') as f:
    # List all groups and datasets in the file
    def print_keys(name, obj):
        print(name)

    f.visititems(print_keys)
    

In [ ]:
# Read, label and sift expression data matrix

sn = serial(5)
chunk_size = 20000 # Define the chunk size

start = time.time()
# Open the HDF5 file in read mode
with h5py.File(read_path, 'r') as file:
    # Access the dataset
    counts = file['data/counts']
    genes = file['data/gene']
    samples = file['data/samples']
    gene_symbols = [str(s)[2:-1] for s in list(genes)]
    # Determine the total number of chunks
    columns = counts.shape[0]
    rows = counts.shape[1]
    num_chunks = rows // chunk_size + (1 if rows % chunk_size else 0)
    print('counts matrix:', columns, 'genes by', rows, 'cells')
    
    print('Read list:', regions)

    for region in regions:
        print('\nReading',region,'data')
        subset_counts = pd.DataFrame()
        # Loop through the chunks
        #for i in range(2):
        for i in range(num_chunks):
            # Calculate the start and end index of the current chunk
            start_index = i * chunk_size
            end_index = min((i + 1) * chunk_size, rows)
            # Read the current chunk from the dataset
            chunk = counts[:,start_index:end_index]
            cell_IDs = [str(s)[2:-1] for s in list(samples[start_index:end_index])]
            # Process chunk
            df = pd.DataFrame(chunk, index=gene_symbols, columns=cell_IDs).T
            raw = df.shape
            # Get rid of non-protein-coding genes
            df = df[pc_genes]
            pc = df.shape

            # Sift desired subset
            sums = df.sum(axis=1)
            genes = df[df > 0].count(axis=1)
            df.index.name = 'sample_name'
            df = annos.merge(df, on='sample_name')            
            if region != 'Union': df = df[df['region'] == region]
            df = df[gene_list]
            if len(df) > 0:
                df.insert(loc=0, column='counts_sum', value=sums)
                df.insert(loc=0, column='genes_mapped', value=genes)
                subset_counts = pd.concat([subset_counts, df])        
            print("%5.1f seconds, " % (time.time()-start),
                  f"Chunk {i+1}/{num_chunks}", raw, pc, df.shape, subset_counts.shape)

        # Merge annotations with counts and update annotation columns
        subset_counts.index.name = 'sample_name'
        df = annos.merge(subset_counts, on='sample_name')
        df.insert(0, 'ordinal', 0)
        df.insert(1, 'type', [type_dict[s] for s in list(df['class'])])
        df['region_order'] = [f"{x:03d}" for x in list( df['region_order'])]
        df['region'] = df['region_order']+' '+df['region']
        df['class_order'] = [f"{x:02d}" for x in list( df['class_order'])]
        df['class'] = df['class_order']+' '+df['class']
        df['neighborhood_order'] = [f"{x:02d}" for x in list( df['neighborhood_order'])]
        df['neighborhood'] = df['neighborhood_order']+' '+df['neighborhood']
        df['subclass_order'] = [f"{x:02d}" for x in list( df['subclass_order'])]
        df['subclass'] = df['subclass_order']+' '+df['subclass']
        df['cluster_order'] = [f"{x:03d}" for x in list( df['cluster_order'])]
        df['cluster'] = [s.split('_')[1] for s in list( df['cluster'])]
        df['cluster'] = df['cluster_order']+' '+df['cluster']

        df.reset_index(inplace=True)
        df.drop(columns=drop_annots, inplace=True)
        df = df[write_annots+gene_list]

        df.set_index(write_annots, inplace=True)
        df.sort_index(level=['type','region','class','neighborhood','subclass','cluster'], inplace=True)
        df.reset_index('ordinal', inplace=True)
        df['ordinal'] = range(len(df))
        df.set_index('ordinal', append=True, inplace=True)

        # Write out support dictionary
        json_path = 'Products/'+dataset+'_'+geneset+'_'+region+'_support_'+sn+'.json'
        print('Writing support data to', json_path)
        with open(json_path, 'w') as fp:
            json.dump(support_dict, fp)

        # Write out trimmed and annotated expression data
        csv_path = 'Products/'+dataset+'_'+geneset+'_'+region+'_counts_'+sn+'.csv.zip'
        print('Writing',region,'data to', csv_path)
        df.to_csv(csv_path)
        end = time.time()

        print('Region',region,"done %5.1f seconds    " % (end-start))

end = time.time()
print("\nFini! %5.1f seconds    " % (end-start))


In [ ]:
# Readback support file
with open(json_path, 'r') as fp:
    temp = json.load(fp)['annotation_labels']
temp


In [ ]:
# Readback expression data
start = time.time()
df = pd.read_csv(csv_path)
end = time.time()
print("Fini! %5.1f seconds    " % (end-start))
# Sanity check
df[['Cck','Npy','Vip','Cckbr','Kng1','Bdkrb1','Bdkrb2']].sum()


In [ ]:
# Set up outputs
json_path = 'Products/'+dataset+'_'+geneset+'_support_'+sn+'.json'
print('              Support path:', json_path)
csv_path = 'Products/'+dataset+'_'+geneset+'_counts_'+sn+'.csv'
print('Expression data write path:', csv_path)


In [ ]:
# Sanity check
print(subset_counts[['Cck','Npy','Vip','Cckbr']].sum())


In [ ]:
# Merge annotations with counts and update annotation columns

df = subset_counts

df.insert(0, 'ordinal', 0)
df.insert(1, 'type', [type_dict[s] for s in list(df['class'])])
df['region_order'] = [f"{x:03d}" for x in list( df['region_order'])]
df['region'] = df['region_order']+' '+df['region']
df['class_order'] = [f"{x:02d}" for x in list( df['class_order'])]
df['class'] = df['class_order']+' '+df['class']
df['neighborhood_order'] = [f"{x:02d}" for x in list( df['neighborhood_order'])]
df['neighborhood'] = df['neighborhood_order']+' '+df['neighborhood']
df['subclass_order'] = [f"{x:02d}" for x in list( df['subclass_order'])]
df['subclass'] = df['subclass_order']+' '+df['subclass']
df['cluster_order'] = [f"{x:03d}" for x in list( df['cluster_order'])]
df['cluster'] = [s.split('_')[1] for s in list( df['cluster'])]
df['cluster'] = df['cluster_order']+' '+df['cluster']

df.reset_index(inplace=True)
df.drop(columns=drop_annots, inplace=True)
df = df[write_annots+gene_list]

df.set_index(write_annots, inplace=True)
df.sort_index(level=['type','region','class','neighborhood','subclass','cluster'], inplace=True)
df.reset_index('ordinal', inplace=True)
df['ordinal'] = range(len(df))
df.set_index('ordinal', append=True, inplace=True)

ann_data = df.copy()


In [ ]:
# Sanity check
print(ann_data[['Cck','Npy','Vip','Cckbr']].sum())


In [ ]:
# Write out support dictionary
with open(json_path, 'w') as fp:
    json.dump(support_dict, fp)
    

In [ ]:
# Write out trimmed and annotated expression data
start = time.time()
ann_data.to_csv(csv_path+'.zip')
end = time.time()
print("\nFini! %5.1f seconds    " % (end-start))


In [ ]:
# json cribs

json_dict = {}

json_dict.update({'missing_pcs': missing_pcs})
json_dict.update({'missing_nMods': missing_nMods})

with open('data.json', 'w') as fp:
    json.dump(json_dict, fp)
    
with open(json_path, 'r') as fp:
    json2_dict = json.load(fp)
    
# json.dump(data, fp, sort_keys=True, indent=4)
    